In [1]:
import pandas as pd
import os
import subprocess
from itertools import product

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!git clone https://github.com/jonahmackey/proxy-a-distance.git
os.chdir(f'./proxy-a-distance')

Cloning into 'proxy-a-distance'...
remote: Enumerating objects: 192, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 192 (delta 107), reused 154 (delta 75), pack-reused 0
Receiving objects: 100% (192/192), 406.37 KiB | 2.27 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [4]:
import main

main.main('./test_data/europarl.en', './test_data/opensubtitles.en', './test_data/vocab')

Training on  1750  examples
[LibLinear]

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Testing on  250  examples
Test MAE:  0.168
PAD value:  1.3279999999999998


# Stuff

In [6]:
langs = ['gu', 'hi', 'ka', 'si', 'ta']
trains = ['cc_align', 'pmo/gov', 'bible']
tests = ['flores', 'bible', 'pmo/gov']
train_sizes = ['0k', '1k', '10k', '25k', '50k', '100k']
test_size = '1k'

In [7]:
lang_map = {
    'gu': 'Gujarati',
    'hi': 'Hindi',
    'ka': 'Kannada',
    'si': 'Sinhala',
    'ta': 'Tamil'
}
gov_map = {
    'gu': 'PrimeMinisterCorpus',
    'hi': 'PrimeMinisterCorpus',
    'ka': 'PrimeMinisterCorpus',
    'si': 'government',
    'ta': 'government'
}
bible_map = {
    'gu': 'new_bible_g1',
    'hi': 'new_bible_g1',
    'ka': 'new_bible_g1',
    'si': 'new_bible_g2',
    'ta': 'new_bible_g1'
}

In [8]:
def get_dataset_name(lang, dataset):
  match dataset:
    case 'cc_align':
      return 'cc_aligned'
    case 'bible':
      return bible_map[lang]
    case 'pmo/gov':
      return gov_map[lang]
    case 'flores':
      return 'flores'

# Create Vocab Files

In [ ]:
def create_vocab(train_path, test_path, vocab_path):
    with open(train_path, 'r') as f:
        train_text = f.read()
    with open(test_path, 'r') as f:
        test_text = f.read()
    vocab = set(train_text.split() + test_text.split())
    os.makedirs(os.path.dirname(vocab_path), exist_ok=True)
    with open(vocab_path, 'w') as f:
        for word in vocab:
            f.write(word + '\n')

In [ ]:
# Create vocab files
for lang, train, train_size, test in product(langs, trains, train_sizes, tests):
  if train_size == '0k':
    pad = 1
  else:
    train_name = get_dataset_name(lang, train)
    test_name = get_dataset_name(lang, test)
    path = 'drive/MyDrive/PerfPred/Dataset/' + lang_map[lang] + '/'
    train_path = path + 'train/' + train_name + '/' + train_size + '/train-en_XX.txt'
    test_path = path + 'test/' + test_name + '/test-en_XX.txt'
    try:
      vocab_dir = path + 'vocab/' + train_name + '/' + train_size + '/' + test_name
      os.makedirs(vocab_dir, exist_ok=True)
      vocab_path = vocab_dir + '/' + 'vocab.txt'
      create_vocab(train_path, test_path, vocab_path)
    except FileNotFoundError:
      continue

# Calculate PAD

In [ ]:
repo_path = 'proxy-a-distance'

In [ ]:
def calculate_pad(train_path, test_path, vocab_path):
  # command = [
  #     'python',
  #     f'{repo_path}/main.py',
  #     train_path,
  #     test_path,
  #     vocab_path
  # ]
  # result = subprocess.run(command, capture_output=True, text=True)
  # print (result.stdout)
  result = main.main(train_path, test_path, vocab_path)
  print (result.stdout)

In [ ]:
df = pd.DataFrame(columns=['language', 'train set', 'train set size',
                           'test set', 'test set size', 'PAD'])
for lang, train, train_size, test in product(langs, trains, train_sizes, tests):
  if train_size == '0k':
    pad = 1
  else:
    train_name = get_dataset_name(lang, train)
    test_name = get_dataset_name(lang, test)
    path = 'drive/MyDrive/PerfPred/Dataset/' + lang_map[lang] + '/'
    train_path = path + 'train/' + train_name + '/' + train_size + '/train-en_XX.txt'
    test_path = path + 'test/' + test_name + '/test-en_XX.txt'
    vocab_path = path + 'vocab/' + train_name + '/' + train_size + '/' + test_name + '/' + 'vocab.txt'

    try:
      calculate_pad (train_path, test_path, vocab_path)
      # TODO: Extract PAD value
    except FileNotFoundError:
      continue

  # row = {'language': lang,
  #       'train set': train,
  #       'train set size': train_size,
  #       'test set': test,
  #       'test set size': test_size,
  #       'PAD': pad}

  # df.loc[len(df.index)] = row
  # df

In [9]:
main.main('/content/drive/MyDrive/PerfPred/Dataset/Gujarati/train/PrimeMinisterCorpus/10k/train-en_XX.txt',
          '/content/drive/MyDrive/PerfPred/Dataset/Gujarati/test/PrimeMinisterCorpus/test-en_XX.txt',
          '/content/drive/MyDrive/PerfPred/Dataset/Gujarati/vocab/PrimeMinisterCorpus/10k/PrimeMinisterCorpus/vocab.txt')

KeyError: ignored